# Analysis Market Price

## 0. Preparation

In [1]:
import pandas as pd

## 1. Data

In [3]:
data = pd.read_csv('push.csv')
print('data.shape:, ', data.shape)
data.head()

data.shape:,  (4, 58)


,Unnamed: 0,OverSellQty,UnderBuyQty,TotalMarketValue,MarketOrderSellQty,MarketOrderBuyQty,BidTime,AskTime,Exchange,ExchangeName,...,PreviousCloseTime,ChangePreviousClose,ChangePreviousClosePer,OpeningPrice,OpeningPriceTime,HighPrice,HighPriceTime,LowPrice,LowPriceTime,SecurityType
0,Price,5236900,2155000,5.661845e+12,0,0,2021-09-15T15:00:00+09:00,2021-09-15T15:00:00+09:00,1,東証１部,...,2021-09-14T00:00:00+09:00,-1.4,-0.19,739.2,2021-09-15T09:00:00+09:00,741.6,2021-09-15T09:54:18+09:00,734.2,2021-09-15T10:54:49+09:00,1
1,Qty,5236900,2155000,5.661845e+12,0,0,2021-09-15T15:00:00+09:00,2021-09-15T15:00:00+09:00,1,東証１部,...,2021-09-14T00:00:00+09:00,-1.4,-0.19,739.2,2021-09-15T09:00:00+09:00,741.6,2021-09-15T09:54:18+09:00,734.2,2021-09-15T10:54:49+09:00,1
2,Sign,5236900,2155000,5.661845e+12,0,0,2021-09-15T15:00:00+09:00,2021-09-15T15:00:00+09:00,1,東証１部,...,2021-09-14T00:00:00+09:00,-1.4,-0.19,739.2,2021-09-15T09:00:00+09:00,741.6,2021-09-15T09:54:18+09:00,734.2,2021-09-15T10:54:49+09:00,1
3,Time,5236900,2155000,5.661845e+12,0,0,2021-09-15T15:00:00+09:00,2021-09-15T15:00:00+09:00,1,東証１部,...,2021-09-14T00:00:00+09:00,-1.4,-0.19,739.2,2021-09-15T09:00:00+09:00,741.6,2021-09-15T09:54:18+09:00,734.2,2021-09-15T10:54:49+09:00,1
